In [ ]:
# Import benötigte Pakete
import os # Dateipfaden
import pickle # Daten zwischenspeichern

# Webscraping 
import undetected_chromedriver as uc # Browser automatisch steuern
from selenium.webdriver.common.by import By # Elemente auf Webseiten finden
from selenium.webdriver.common.action_chains import ActionChains # Interaktionen auf Webseiten 
from selenium.webdriver.support.ui import WebDriverWait # Warten auf Elemente
from selenium.webdriver.support import expected_conditions as EC # Bedingungen für Webscraping
import time # Zeitsteuerung

# Bearbeiten von html-Dateien
from bs4 import BeautifulSoup  # HTML auslesen und bereinigen
import requests # HTTP-Anfragen

In [ ]:
# Pfade 

# Input 
LINKS_FILE = os.path.join("input", "pm_bvg_raw", "links.pkl") # Pfad für Links
SAVE_FOLDER = os.path.join("input", "pm_bvg_raw") # Pfad für html-Dateien
os.makedirs(SAVE_FOLDER, exist_ok=True)

# Output
OUTPUT_FOLDER = os.path.join("output") # Ordner für Output
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
CSV_PATH = os.path.join("output", "pm_bvg_valid.csv") # Pfad für CSV-Datei
SQL_PATH = os.path.join(OUTPUT_FOLDER, "dwh_pm_bvg.sqlite3") # Pfad für SQL-Datenbank

# Stoppwörter einmalig laden
STOPWORDS_URL = "https://raw.githubusercontent.com/solariz/german_stopwords/master/german_stopwords_full.txt"
stopwords_list = requests.get(STOPWORDS_URL, allow_redirects=True).text.split("\n")[9:]


In [ ]:
# Scraping Pressemitteilungs-Links BVG-Webseite
# Wenn Datei existiert → nicht nochmal scrapen
if os.path.exists(LINKS_FILE):
    print("Links wurden bereits gesammelt")
    with open(LINKS_FILE, "rb") as f:
        all_links = pickle.load(f)

# Wenn nein → Webscraping starten
else:
    print("Starte Webscraping...")

 # Browser öffnen
    driver = uc.Chrome()
    driver.get("https://www.bvg.de/de/unternehmen/medienportal/pressemitteilungen")
    time.sleep(5)

    all_links = set() # Leeres Set für Links
    current_page = 1 # Start bei Seite 1

    while True:
        print(f"Seite {current_page}")
        time.sleep(3) # Pause damit die Seite vollständig lädt

        # Links sammeln
        links = driver.find_elements(By.CSS_SELECTOR, "a[href^='/de/unternehmen/medienportal/pressemitteilungen/']") # Mit CSS-Selektor alle Links finden
        for link in links:
            href = link.get_attribute("href")
            if href:
                all_links.add(href)

        try:
            # Versuchen, die nächste Seite zu finden
            next_page_number = str(current_page + 1)
            wait = WebDriverWait(driver, 10) # Maximal 10 Sekunden warten
            next_button = wait.until(EC.presence_of_element_located( # Warten, bis der Button für die nächste Seite sichtbar ist
                (By.XPATH, f"//button[normalize-space(text())='{next_page_number}']"))) # Mit XPATH den Button für die nächste Seite finden

            # Zum Button scrollen und klicken
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
            time.sleep(1)
            ActionChains(driver).move_to_element(next_button).click().perform()

            # Seitenzähler erhöhen und kleine Pause
            current_page += 1
            time.sleep(4)

        except Exception as e:
             # Falls keine weitere Seite existiert oder ein Fehler auftritt → Schleife beenden
            print(f"Keine weitere Seite gefunden oder Fehler bei Seite {current_page}: {e}")
            break

    # Browser schließen
    driver.quit()

    # Links speichern
    os.makedirs(os.path.dirname(LINKS_FILE), exist_ok=True)
    with open(LINKS_FILE, "wb") as f:
        pickle.dump(all_links, f)

    print(f"Links gespeichert ({len(all_links)} Stück).")

# Gefundene Links anzeigen
print("\nGefundene Links:")
for link in all_links:
    print(link)

Links wurden bereits gesammelt. Lade sie...

Gefundene Links:
https://www.bvg.de/de/unternehmen/medienportal/pressemitteilungen/das-sind-die-neuen-die-kommen-jetzt-oefter
https://www.bvg.de/de/unternehmen/medienportal/pressemitteilungen/wir-uebernehmen-die-leitung
https://www.bvg.de/de/unternehmen/medienportal/pressemitteilungen/bvg-ruestet-auf
https://www.bvg.de/de/unternehmen/medienportal/pressemitteilungen/frisch-geliftet
https://www.bvg.de/de/unternehmen/medienportal/pressemitteilungen/dichter-und-denker
https://www.bvg.de/de/unternehmen/medienportal/pressemitteilungen/hoer-mal-wer-da-spricht
https://www.bvg.de/de/unternehmen/medienportal/pressemitteilungen/besserwissen-erlaubt
https://www.bvg.de/de/unternehmen/medienportal/pressemitteilungen/scheibenkleister
https://www.bvg.de/de/unternehmen/medienportal/pressemitteilungen/zurueck-in-die-zukunft-2
https://www.bvg.de/de/unternehmen/medienportal/pressemitteilungen/das-beste-zum-abschluss
https://www.bvg.de/de/unternehmen/medienporta

In [ ]:
# Check Pickle-Date
print(os.path.exists("input/pm_bvg_raw/links.pkl"))

True


In [ ]:
# Check Anzahl Links
print(f"Anzahl gesammelter Links: {len(all_links)}")

Anzahl gesammelter Links: 910


In [ ]:
# Links laden
with open(LINKS_FILE, "rb") as f:
    all_links = pickle.load(f)

print(f"Anzahl gesammelter Links: {len(all_links)}")

Anzahl gesammelter Links: 910
⏩ Bereits vorhanden, überspringe: das-sind-die-neuen-die-kommen-jetzt-oefter.html
⏩ Bereits vorhanden, überspringe: wir-uebernehmen-die-leitung.html
⏩ Bereits vorhanden, überspringe: bvg-ruestet-auf.html
⏩ Bereits vorhanden, überspringe: frisch-geliftet.html
⏩ Bereits vorhanden, überspringe: dichter-und-denker.html
⏩ Bereits vorhanden, überspringe: hoer-mal-wer-da-spricht.html
⏩ Bereits vorhanden, überspringe: besserwissen-erlaubt.html
⏩ Bereits vorhanden, überspringe: scheibenkleister.html
⏩ Bereits vorhanden, überspringe: zurueck-in-die-zukunft-2.html
⏩ Bereits vorhanden, überspringe: das-beste-zum-abschluss.html
⏩ Bereits vorhanden, überspringe: mueggelseetram-2.html
⏩ Bereits vorhanden, überspringe: 2025-01-15-tarif-2025.html
⏩ Bereits vorhanden, überspringe: nachtschicht-unterm-adenauerplatz.html
⏩ Bereits vorhanden, überspringe: da-kehrt-flexibilitaet-ein.html
⏩ Bereits vorhanden, überspringe: wird-gleis-erledigt.html
⏩ Bereits vorhanden, überspringe

In [ ]:
# html-Dateien herunterladen
for url in all_links:
    try:
        # Dateiname erstellen aus dem letzten Teil der URL
        slug = url.strip("/").split("/")[-1]
        filename = f"{slug}.html"
        save_path = os.path.join(SAVE_FOLDER, filename)

        # Nur herunterladen, wenn Datei noch nicht existiert
        if os.path.exists(save_path):
            print(f"Bereits vorhanden, überspringe: {filename}")
            continue

        # html-Inhalt herunterladen
        response = requests.get(url)
        response.raise_for_status()  # Fehler werfen, wenn Download fehlschlägt

        # Inhalt speichern
        with open(save_path, "w", encoding="utf-8") as f_out:
            f_out.write(response.text)

        print(f"Gespeichert: {filename}")

    except Exception as e:
        print(f"Fehler bei {url}: {e}")

In [ ]:
# Alle Dateien in diesem Ordner auflisten
html_files = [f for f in os.listdir(SAVE_FOLDER) if f.endswith(".html")]

# Ergebnis anzeigen
print(f"Anzahl gespeicherter html-Dateien: {len(html_files)}")

# Beispielhafte Dateinamen ausgeben
print("Beispieldateien:", html_files[:5])

📄 Anzahl gespeicherter HTML-Dateien: 758
Beispieldateien: ['10-uhr-karten-gelten-eine-stunde-frueher.html', '13-points-go-to.html', '140-jahre-unter-strom.html', '155-000-mal-5.html', '17-000-kostenlose-fahrscheine-s-bahn-berlin-und-bvg-unterstuetzen-ehrenamtliche.html']


In [ ]:
# Check Links
files = os.listdir("input/pm_bvg_raw")
print(len(files))
print(files[:5])

759
['10-uhr-karten-gelten-eine-stunde-frueher.html', '13-points-go-to.html', '140-jahre-unter-strom.html', '155-000-mal-5.html', '17-000-kostenlose-fahrscheine-s-bahn-berlin-und-bvg-unterstuetzen-ehrenamtliche.html']


In [ ]:
# Liste für die Daten
data = []

# Alle HTML-Dateien durchgehen
files = [f for f in os.listdir(SAVE_FOLDER) if f.endswith(".html")]

for file in files:
    file_path = os.path.join(SAVE_FOLDER, file)
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            html = f.read()

        soup = BeautifulSoup(html, "html.parser") # Links mit BeautifulSoup parsen
        time_tag = soup.find("time") # Time-Element suchen

        if time_tag and time_tag.has_attr("datetime"): # Prüfen, ob Veröffentlichungsdatum vorhanden
            date_str = time_tag["datetime"][:10]  # # Nur Datumsformat yyyy-mm-dd 
            year = int(date_str.split("-")[0]) # Jahr extrahieren

            # Pressemitteilungen 2021–2025 übernehmen
            if 2021 <= year <= 2025:
                data.append({
                    "filename": file,
                    "date": date_str,
                    "year": year
                })
            else:
                # Dateien aus anderen Jahren ignorieren
                print(f"Datei {file} aus Jahr {year}, wird ignoriert.")
        else:
            print(f"Kein Datum gefunden in {file}")

    except Exception as e:
        print(f"Fehler bei {file}: {e}")

# DataFrame erstellen
pm_df = pd.DataFrame(data)

⏩ Datei 10-uhr-karten-gelten-eine-stunde-frueher.html aus Jahr 2020, wird ignoriert.
⏩ Datei 17-000-kostenlose-fahrscheine-s-bahn-berlin-und-bvg-unterstuetzen-ehrenamtliche.html aus Jahr 2018, wird ignoriert.
⏩ Datei 21-geteilt-durch-zwei.html aus Jahr 2018, wird ignoriert.
⏩ Datei 3-2-1-los.html aus Jahr 2018, wird ignoriert.
⏩ Datei 90-baeume-fuer-berlin.html aus Jahr 2019, wird ignoriert.
⏩ Datei a1-auf-u3.html aus Jahr 2018, wird ignoriert.
⏩ Datei abgeleitet.html aus Jahr 2019, wird ignoriert.
⏩ Datei ach-du-jelbe-neune.html aus Jahr 2020, wird ignoriert.
⏩ Datei ach-gleis-drauf.html aus Jahr 2020, wird ignoriert.
⏩ Datei aehm-2.html aus Jahr 2020, wird ignoriert.
⏩ Datei alexanderpendel-bis-fraesfurter-allee.html aus Jahr 2018, wird ignoriert.
⏩ Datei alle-jahre-wieder-2.html aus Jahr 2020, wird ignoriert.
⏩ Datei alle-jahre-wieder.html aus Jahr 2018, wird ignoriert.
⏩ Datei alle-wege-fuehren-nach-strom.html aus Jahr 2019, wird ignoriert.
⏩ Datei aller-guten-dinge-sind-vier.html 

In [ ]:
# Metadaten als CSV-Datei speichern
csv_path = os.path.join(OUTPUT_FOLDER, "pm_bvg_valid.csv")
pm_df.to_csv(csv_path, index=False)
print(f"CSV gespeichert unter {csv_path}")

✅ CSV gespeichert unter output\pm_bvg_valid.csv


In [ ]:
# Check CSV 
if os.path.exists(CSV_PATH):
    # CSV laden
    df = pd.read_csv(CSV_PATH)
    
    # Erste Zeilen anzeigen
    print("Erste Zeilen der CSV-Datei:")
    print(df.head())
else:
    print("CSV-Datei wurde nicht gefunden. Pfad und Dateiname prüfen.")

✅ Erste Zeilen der CSV-Datei:
                                            filename        date  year
0                               13-points-go-to.html  2021-08-06  2021
1                         140-jahre-unter-strom.html  2021-05-12  2021
2                                 155-000-mal-5.html  2021-12-01  2021
3                         2025-01-15-tarif-2025.html  2025-04-10  2025
4  2025-01-29-pm-bvg-muva-barrierefrei-ab-maerz-b...  2025-01-29  2025


In [ ]:
# Metadaten als SQL speichern
SQL_PATH = os.path.join(OUTPUT_FOLDER, "dwh_pm_bvg.sqlite3")
conn = sqlite3.connect(SQL_PATH)
pm_df.to_sql("pm_bvg_valid", conn, if_exists="replace", index=False)
conn.close()
print(f"SQL gespeichert unter {sql_path}")

✅ SQL gespeichert unter output\dwh_pm_bvg.sqlite3


In [ ]:
# Encoding prüfen
file_path = os.path.join(SAVE_FOLDER, "3-2-1-los.html")  # Richtiger Pfad zur Datei
with open(file_path, "r", encoding="utf-8") as f:
    html = f.read()
    print(html[:500])  

In [ ]:
# html-Datei mit Encoding einlesen 
def read_html_file(filepath, encoding="utf-8"):
    # Datei öffnen und einlesen
    with open(filepath, "r", encoding=encoding) as f:
        return f.read()

# html-Inhalt bereinigen & in einzelne Wörter zerlegen 
def process_html(html):
    # html-Parser: nur sichtbaren Text auslesen
    bstext = BeautifulSoup(html, "html.parser")
    text = bstext.get_text(separator=" ").lower()

    # Zeilenumbrüche durch Leerzeichen ersetzen & Wörter trennen
    items = text.replace("\n", " ").split(" ")

    # Kurze Wörter und Stoppwörter entfernen
    items = [i for i in items if len(i) > 1 and i not in stopwords_list]
    return items

In [49]:
# Einzelne Pressemitteilungsdatei verarbeiten
def process_press_release(press_release):
    # Nur Dateinamen rausholen
    filename = os.path.basename(press_release["filename"])
    full_path = os.path.join(SAVE_FOLDER, filename)

    # Pressemitteilungen sind UTF-8 codiert 
    html = read_html_file(full_path, encoding="utf-8")
    items = process_html(html)

    # Häufigkeit jedes Wortes zählen
    count = pd.Series(items).value_counts()

    # Als DataFrame formatieren und Zusatzinfos ergänzen
    count_df = count.to_frame()
    count_df.columns = ["count"]
    count_df["word"] = count_df.index
    count_df["source"] = "bvg_pm"  # Quelle festlegen (z.B. bvg_pm = Pressemitteilung)
    count_df["date"] = press_release["date"]

    return count_df

In [50]:
# Liste für die Ergebnisse-Sammlung 
collection = []

In [51]:
# Verarbeitung jeder einzelnen Pressemitteilung
def process_wrapper(press_release):
    filename = press_release["filename"]
    try:
        count = process_press_release(press_release)
        print(f"[INFO] Verarbeitung erfolgreich: {filename}")
        collection.append(count)
    except Exception as e:
        print(f"[ERROR] Fehler bei {filename}: {e}")

In [52]:
# Verarbeitung aller Pressemitteilungen
pm_df.apply(process_wrapper, axis=1)

# --- Alle Ergebnisse in einer Tabelle zusammenführen ---
data = pd.concat(collection, axis=0)
print("Data shape:", data.shape)

[INFO] Verarbeitung erfolgreich: 13-points-go-to.html
[INFO] Verarbeitung erfolgreich: 140-jahre-unter-strom.html
[INFO] Verarbeitung erfolgreich: 155-000-mal-5.html
[INFO] Verarbeitung erfolgreich: 2025-01-15-tarif-2025.html
[INFO] Verarbeitung erfolgreich: 2025-01-29-pm-bvg-muva-barrierefrei-ab-maerz-berlinweit.html
[INFO] Verarbeitung erfolgreich: 2025-02-04-ideen-die-berlin-bewegen.html
[INFO] Verarbeitung erfolgreich: 2025-02-19-aufsichtsrat-beschliesst-neuaufstellung-des-bvg-vorstands.html
[INFO] Verarbeitung erfolgreich: 2025-03-10-update-stabilitaet-der-bvg0.html
[INFO] Verarbeitung erfolgreich: 2025-03-13-neuer-betriebshof.html
[INFO] Verarbeitung erfolgreich: 2025-03-28-faehr-geht-vor-.html
[INFO] Verarbeitung erfolgreich: 2025-04-02-abschluss-kleidferspende.html
[INFO] Verarbeitung erfolgreich: 2025-04-14-abschluss-reinigungsstreife.html
[INFO] Verarbeitung erfolgreich: 2025-04-16-neue-linie-215.html
[INFO] Verarbeitung erfolgreich: 2025-04-17-ankuendigung-moblitaetstraining

In [ ]:
# Wortzählung als SQL speichern

# Verbindung zur Datenbank aufbauen 
conn = sqlite3.connect(SQL_PATH)

# Wortzählung speichern 
data.to_sql("wordcount_pm", conn, if_exists="replace", index=False)

# Verbindung schließen
conn.close()

print(f"SQL gespeichert unter {SQL_PATH}")

SQL gespeichert unter output\dwh_pm_bvg.sqlite3


In [ ]:
# Check Tabellen in Datenbank
# Verbindung zur Datenbank aufbauen
conn = sqlite3.connect(os.path.join(OUTPUT_FOLDER, "dwh_pm_bvg.sqlite3"))

# Cursor-Objekt erstellen
cursor = conn.cursor()

# Abfrage: Alle Tabellennamen aus der Datenbank holen
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Ergebnis holen
tables = cursor.fetchall()

# Tabellen ausgeben
print("Tabellen in der Datenbank:")
for table in tables:
    print(table[0])

# Verbindung schließen
conn.close()

Tabellen in der Datenbank:
pm_bvg_valid
wordcount_pm
